In [ ]:
import math
import torch
import torch.nn as nn

import pandas as pd
import seaborn as sns
import numpy as np

import matplotlib.pyplot as plt
from collections import OrderedDict

sns.set_style("white")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
inputs = np.arange(21,101,1)
outputs = np.zeros((80, 1803))
for Re in range(21,101,1):
    outputs[Re] = np.loadtxt('data/weights/weights_'+str(Re)+'.txt')

In [ ]:
class HyperNetwork(nn.Module):
    def __init__(self):
        super(HyperNetwork, self).__init__()
        
        self.layers = nn.Sequential(nn.Linear(1,512),
                      nn.ReLU(),             
                      nn.Linear(512,256),
                      nn.ReLU(),
                      nn.Linear(256,256),
                      nn.ReLU(),
                      nn.Linear(256,128),
                      nn.ReLU(),
                      nn.Linear(128,1803))    
        
    def forward(self, Re):        
        weights = self.layers(Re)
        return weights

In [ ]:
hnet = HyperNetwork()
hnet = hnet.to(device)
mse_cost_function = nn.MSELoss() 
optimizer = torch.optim.Adam(hnet.parameters())

In [ ]:
iterations = 5000
for epoch in range(iterations):
    optimizer.zero_grad()
    
    hnet_out = hnet.forward(inputs)
    Loss = mse_cost_function(hnet_out, outputs)
    Loss.backward()
    optimizer.step()

    with torch.autograd.no_grad():
        if epoch%100 == 0:
            print('Iter %d, Loss: %.4e, Loss_u: %.4e, Loss_f: %.4e' % (epoch, Loss))